In [1]:
import pandas as pd
from sqlalchemy import create_engine
from datetime import datetime, timedelta

# 設置MySQL資料庫連接
db_user = 'root'
db_password = '19970730'
db_host = '127.0.0.1'
db_name = 'sql_stock'

# 創建資料庫連接引擎
engine = create_engine(f'mysql+mysqlconnector://{db_user}:{db_password}@{db_host}/{db_name}')

# 設置日期範圍
#today = datetime(2024, 5, 25)
today = datetime.now().date()
one_year_ago = today - timedelta(days=720)

# 格式化日期
today_str = today.strftime('%Y%m%d')
one_year_ago_str = one_year_ago.strftime('%Y%m%d')

# 全撈 
one_year_ago_str = '20210101'

# SQL 查詢 加權指數
query_twse = f"""
SELECT *
FROM daily_twse
WHERE date >= '{one_year_ago_str}' AND date <= '{today_str}'
"""

# SQL 查詢 個股
query_stock = f"""
SELECT *
FROM daily_price
WHERE date >= '{one_year_ago_str}' AND date <= '{today_str}'
"""

# SQL 查詢 細產業
query_sub_category = f"""
SELECT *
FROM sub_category
"""

# 使用 pd.read_sql 來執行查詢並讀取數據到 DataFrame
stock_df_raw = pd.read_sql(query_stock, engine)


In [2]:
import pandas as pd
from sqlalchemy import create_engine
from datetime import datetime, timedelta

# 設置MySQL資料庫連接
db_user = 'root'
db_password = '19970730'
db_host = '127.0.0.1'
db_name = 'sql_stock'

# 創建資料庫連接引擎
engine = create_engine(f'mysql+mysqlconnector://{db_user}:{db_password}@{db_host}/{db_name}')


# SQL 查詢 細產業
query_sub_category = f"""
SELECT *
FROM sub_category
"""

# 使用 pd.read_sql 來執行查詢並讀取數據到 DataFrame

sub_category_list = pd.read_sql(query_sub_category, engine)



In [26]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

daily_tx_data = pd.DataFrame()

# 目標 URL
url_tx = "https://www.taifex.com.tw/cht/3/futDailyMarketReport"
queryDate = '2024/09/03'

# POST 表單數據
payload_tx = {
    'queryType': '1',            # 每日報告
    'marketCode': '0',           # 市場代碼，0 表示全部
    'commodity_id': 'TX',        # 商品代碼，TX 代表臺股期貨
    'queryDate': queryDate,     # 查詢日期，格式為 YYYYMMDD
    'MarketCode': '0'
}

# 發送 POST 請求
response = requests.post(url_tx, data=payload_tx, timeout=10)

# 檢查請求是否成功
if response.status_code == 200:
    # 解析 HTML
    soup = BeautifulSoup(response.text, 'html.parser')

    # 找到數據表格
    table = soup.find('table', class_='table_f')

    # 提取表格中的數據
    rows = []
    for row in table.find_all('tr')[1:]:  # 跳過表頭行
        cols = row.find_all('td')
        cols = [col.text.strip() for col in cols]
        rows.append(cols)

else:
    print(f"請求失敗，狀態碼：{response.status_code}")

if (rows[0][0] == 'TX'):
    daily_tx_data = pd.DataFrame({
        'Date': [queryDate], 
        '契約': [rows[0][0]],
        '到期月份': [rows[0][1]],
        '開盤價': [rows[0][2]],
        '最高價': [rows[0][3]],
        '最低價': [rows[0][4]],
        '最後成交價': [rows[0][5]],
        '一般交易時段成交量': [rows[0][9]]
    })

daily_tx_data


,Date,契約,到期月份,開盤價,最高價,最低價,最後成交價,一般交易時段成交量
0,2024/09/03,TX,202409,22205,22314,22000,22070,72699


In [28]:
import requests
import re
from bs4 import BeautifulSoup
import pandas as pd

tif_investors_data = pd.DataFrame()

# 設定目標 URL 和表單提交參數
url_top5 = "https://www.taifex.com.tw/cht/3/largeTraderFutQry"
queryDate = '2024/09/04'

# 表單數據
payload_top5 = {
    'queryDate': queryDate,               
    'contractId': 'TX'  # 臺股期貨+小臺指期貨/4+臺指摩根期貨/20
}


tx_top5 = 0
# 發送 POST 請求
response = requests.post(url_top5, data=payload_top5)

# 確認請求是否成功
if response.status_code == 200:
    # 解析 HTML 結果
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # 提取目標表格
    table = soup.find('table', class_='table_f')

    # 解析表格內容
    rows = []
    for row in table.find_all('tr')[1:]:  # 跳過標題行
        cols = row.find_all('td')
        cols = [col.text.strip() for col in cols]
        rows.append(cols)

else:
    print(f"請求失敗，狀態碼：{response.status_code}")

if((rows[2][0] == '臺股期貨(TX+MTX/4+TMF/20)') & (rows[4][0] == '所有契約')):
    text = rows[4][1]
    match = re.search(r'\(([\d,]+)\)', text)
    # 如果找到匹配，去除逗號並取出數字
    if match:
        tx_top5 = match.group(1).replace(',', '')


# 設定目標 URL 和表單提交的參數（包括日期）
url_tif = "https://www.taifex.com.tw/cht/3/futContractsDate"

# 你可以更改日期來獲取不同天數的數據
payload_tif = {
    'queryDate': queryDate,  # 若要查詢特定日期，填入日期
    'commodity_id': 'TXF'  # TX 表示臺股期貨
}


# 發送 POST 請求
response = requests.post(url_tif, data=payload_tif)

# 檢查請求是否成功
if response.status_code == 200:
    # 解析 HTML
    soup = BeautifulSoup(response.text, 'html.parser')
    trust_data = []    
    # 尋找數據表格

    # 尋找數據表格
    table = soup.find('table', class_='table_f')
    if table:
        rows = table.find_all("tr")[1:]  # 忽略表头
        for row in rows:
            columns = row.find_all("td")
            row_data = [col.text.strip() for col in columns]
            if(len(row_data)!=0):
                trust_data.append(row_data)

    if ( (trust_data[0][1] == '臺股期貨') &(trust_data[0][2] == '自營商') & (trust_data[1][0] == '投信') & (trust_data[2][0] == '外資')):

        tif_investors_data = pd.DataFrame({
            'Date': [queryDate], 
            '自營未平倉餘額口數': [trust_data[0][13].replace(',', '')],
            '自營未平倉餘額金額': [trust_data[0][14].replace(',', '')],
            '投信未平倉餘額口數': [trust_data[1][11].replace(',', '')],
            '投信未平倉餘額金額': [trust_data[1][12].replace(',', '')],
            '外資未平倉餘額口數': [trust_data[2][11].replace(',', '')],
            '外資未平倉餘額金額': [trust_data[2][12].replace(',', '')],
            '前五大交易人合計': [tx_top5]
    })
    
else:
    print(f"請求失敗，狀態碼：{response.status_code}")


tif_investors_data

,Date,自營未平倉餘額口數,自營未平倉餘額金額,投信未平倉餘額口數,投信未平倉餘額金額,外資未平倉餘額口數,外資未平倉餘額金額,前五大交易人合計
0,2024/09/04,1203,5036338,22426,93920072,-30917,-129479963,33213


In [69]:
import requests
import re
from bs4 import BeautifulSoup
import pandas as pd

# 設定目標 URL 和表單提交參數
url_top5 = "https://www.taifex.com.tw/cht/3/largeTraderFutQry"

# 表單數據
payload_top5 = {
    'queryDate': '2024/09/02',               
    'contractId': 'TX'  # 臺股期貨+小臺指期貨/4+臺指摩根期貨/20
}


tx_top5 = 0
# 發送 POST 請求
response = requests.post(url_top5, data=payload_top5)

# 確認請求是否成功
if response.status_code == 200:
    # 解析 HTML 結果
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # 提取目標表格
    table = soup.find('table', class_='table_f')

    # 解析表格內容
    rows = []
    for row in table.find_all('tr')[1:]:  # 跳過標題行
        cols = row.find_all('td')
        cols = [col.text.strip() for col in cols]
        rows.append(cols)

else:
    print(f"請求失敗，狀態碼：{response.status_code}")

if((rows[2][0] == '臺股期貨(TX+MTX/4+TMF/20)') & (rows[4][0] == '所有契約')):
    text = rows[4][1]
    match = re.search(r'\(([\d,]+)\)', text)
    # 如果找到匹配，去除逗號並取出數字
    if match:
        tx_top5 = match.group(1).replace(',', '')
    

35722


In [78]:
rows[4][0]

'所有契約'